In [11]:
import sys
sys.path.append('/Users/jsimpson/Projects/personal/python_scripts/fpl_predictor')
import os
os.chdir('/Users/jsimpson/Projects/personal/python_scripts/fpl_predictor')
import master

In [12]:
import pandas as pd
fixtures_df = pd.read_csv('/Users/jsimpson/Projects/personal/python_scripts/fpl_predictor/data_fpl/data/2023-24/fixtures.csv')
fixtures_df.columns
# fixtures_df['team_h']


Index(['code', 'event', 'finished', 'finished_provisional', 'id',
       'kickoff_time', 'minutes', 'provisional_start_time', 'started',
       'team_a', 'team_a_score', 'team_h', 'team_h_score', 'stats',
       'team_h_difficulty', 'team_a_difficulty', 'pulse_id'],
      dtype='object')

In [13]:
merged_gw_df = pd.read_csv('/Users/jsimpson/Projects/personal/python_scripts/fpl_predictor/data_fpl/data/2023-24/gws/merged_gw.csv')
# merged_gw_df.head()
merged_gw_df.columns

Index(['name', 'position', 'team', 'xP', 'assists', 'bonus', 'bps',
       'clean_sheets', 'creativity', 'element', 'expected_assists',
       'expected_goal_involvements', 'expected_goals',
       'expected_goals_conceded', 'fixture', 'goals_conceded', 'goals_scored',
       'ict_index', 'influence', 'kickoff_time', 'minutes', 'opponent_team',
       'own_goals', 'penalties_missed', 'penalties_saved', 'red_cards',
       'round', 'saves', 'selected', 'starts', 'team_a_score', 'team_h_score',
       'threat', 'total_points', 'transfers_balance', 'transfers_in',
       'transfers_out', 'value', 'was_home', 'yellow_cards', 'GW'],
      dtype='object')

In [14]:
merged_gw_df["fixture"]

0          7
1          4
2         10
3          7
4          8
        ... 
29720    379
29721    378
29722    376
29723    374
29724    372
Name: fixture, Length: 29725, dtype: int64

In [15]:
mohamed_salah_df = merged_gw_df[merged_gw_df['name'] == 'Mohamed Salah']
mohamed_salah_df.head()

,name,position,team,xP,assists,bonus,bps,clean_sheets,creativity,element,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,fixture,goals_conceded,goals_scored,ict_index,influence,kickoff_time,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,round,saves,selected,starts,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
551,Mohamed Salah,MID,Liverpool,4.5,1,0,15,0,20.2,308,0.31,0.59,0.28,1.28,9,1,0,4.9,16.6,2023-08-13T15:30:00Z,76,7,0,0,0,0,1,0,2243674,1,1,1,12.0,5,0,0,0,125,False,0,1
1158,Mohamed Salah,MID,Liverpool,5.5,0,0,15,0,33.0,308,0.11,1.34,1.23,1.00,14,1,1,15.3,41.4,2023-08-19T14:00:00Z,85,3,0,1,0,0,2,0,2739755,1,1,3,79.0,5,163064,211617,48553,125,True,0,2
1825,Mohamed Salah,MID,Liverpool,5.0,1,0,19,0,24.9,308,0.12,0.30,0.18,2.00,29,1,0,7.0,28.6,2023-08-27T15:30:00Z,90,15,0,0,0,0,3,0,2570086,1,2,1,16.0,5,-229297,98738,328035,125,False,0,3
2525,Mohamed Salah,MID,Liverpool,6.3,0,0,26,1,23.1,308,0.07,0.87,0.80,0.66,37,0,1,18.3,45.2,2023-09-03T13:00:00Z,90,2,0,0,0,0,4,0,2505698,1,0,3,115.0,8,-122416,74612,197028,125,True,0,4
3239,Mohamed Salah,MID,Liverpool,8.2,2,2,35,0,57.4,308,1.47,1.71,0.24,0.63,50,1,0,12.1,44.2,2023-09-16T11:30:00Z,90,20,0,0,0,0,5,0,2540130,1,3,1,19.0,10,-22644,117182,139826,125,False,0,5


In [68]:
# Merge when was_home is True
home_merge = pd.merge(mohamed_salah_df[mohamed_salah_df['was_home'] == True], fixtures_df, how='left', left_on=['kickoff_time', 'opponent_team'], right_on=['kickoff_time', 'team_a'])

# Merge when was_home is False
away_merge = pd.merge(mohamed_salah_df[mohamed_salah_df['was_home'] == False], fixtures_df, how='left', left_on=['kickoff_time', 'opponent_team'], right_on=['kickoff_time', 'team_h'])

# Combine the results
merged_df = pd.concat([home_merge, away_merge])

# Drop duplicate columns after merge
# merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]
print(len(merged_df))

38


In [74]:
pd.set_option('display.max_columns', None)
merged_df = merged_df.sort_values(by='GW')
print(merged_df.head())

            name position       team   xP  assists  bonus  bps  clean_sheets  \
0  Mohamed Salah      MID  Liverpool  4.5        1      0   15             0   
0  Mohamed Salah      MID  Liverpool  5.5        0      0   15             0   
1  Mohamed Salah      MID  Liverpool  5.0        1      0   19             0   
1  Mohamed Salah      MID  Liverpool  6.3        0      0   26             1   
2  Mohamed Salah      MID  Liverpool  8.2        2      2   35             0   

   creativity  element  expected_assists  expected_goal_involvements  \
0        20.2      308              0.31                        0.59   
0        33.0      308              0.11                        1.34   
1        24.9      308              0.12                        0.30   
1        23.1      308              0.07                        0.87   
2        57.4      308              1.47                        1.71   

   expected_goals  expected_goals_conceded  fixture  goals_conceded  \
0            0.

In [90]:
# Check if the columns exist
fdr = [] 
for idx, row in merged_df.iterrows():
	if row["was_home"] == True:
		fdr.append(row["team_h_difficulty"])
	else:
		fdr.append(row["team_a_difficulty"])
	
merged_df["fdr"] = fdr
print(merged_df[["GW", "fdr"]])

    GW  fdr
0    1    3
0    2    2
1    3    4
1    4    3
2    5    2
2    6    2
3    7    4
4    8    3
3    9    2
4   10    2
5   11    2
5   12    2
6   13    5
6   14    2
7   15    2
8   16    2
7   17    3
8   18    5
9   19    2
9   20    3
10  21    2
10  22    3
11  23    5
11  24    2
12  25    2
12  25    2
13  27    2
13  28    5
14  30    2
15  31    2
14  32    4
16  33    2
16  34    2
15  34    3
17  35    3
17  36    3
18  37    4
18  38    2


In [92]:
fixtures_df_25 = pd.read_csv('/Users/jsimpson/Projects/personal/python_scripts/fpl_predictor/data_fpl/data/2024-25/fixtures.csv')
fixtures_df_25.tail()


,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,pulse_id
375,2444845,38.0,False,False,376,2025-05-25T15:00:00Z,0,False,False,8,NaN,15,NaN,[],2,4,116202
376,2444846,38.0,False,False,377,2025-05-25T15:00:00Z,0,False,False,6,NaN,16,NaN,[],4,3,116203
377,2444847,38.0,False,False,378,2025-05-25T15:00:00Z,0,False,False,1,NaN,17,NaN,[],4,2,116204
378,2444848,38.0,False,False,379,2025-05-25T15:00:00Z,0,False,False,5,NaN,18,NaN,[],3,4,116205
379,2444849,38.0,False,False,380,2025-05-25T15:00:00Z,0,False,False,4,NaN,20,NaN,[],2,2,116206
